. goals made || attk score   
. goals allowed || defeense score   
. games score (lose-0 | tie-1 | win-3)    

. head 2 head (season | last2 seasons | historical)   

. LAST X GAMES

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
df = pd.read_csv('../data/raw/dataset.csv', parse_dates=['date'])
df

,fixture_id,date,league_season,league,home_team,away_team,home_score,away_score,home_result
0,720746,2021-08-13 19:00:00+00:00,2021,La Liga,Valencia,Getafe,1,0,WIN
1,720743,2021-08-14 17:30:00+00:00,2021,La Liga,Cadiz,Levante,1,1,DRAW
2,720742,2021-08-14 17:30:00+00:00,2021,La Liga,Mallorca,Real Betis,1,1,DRAW
3,720739,2021-08-14 20:00:00+00:00,2021,La Liga,Alaves,Real Madrid,1,4,LOSS
4,720744,2021-08-14 20:00:00+00:00,2021,La Liga,Osasuna,Espanyol,0,0,DRAW
...,...,...,...,...,...,...,...,...,...
1135,1038330,2024-05-25 19:00:00+00:00,2023,La Liga,Real Madrid,Real Betis,0,0,DRAW
1136,1038324,2024-05-26 12:00:00+00:00,2023,La Liga,Getafe,Mallorca,1,2,LOSS
1137,1038331,2024-05-26 14:15:00+00:00,2023,La Liga,Las Palmas,Alaves,1,1,DRAW
1138,1038323,2024-05-26 14:15:00+00:00,2023,La Liga,Celta Vigo,Valencia,2,2,DRAW


In [46]:
# df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
# df['date'] = pd.to_datetime(df['date']).dt.strftime('%d-%m-%Y')

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   fixture_id     1140 non-null   int64              
 1   date           1140 non-null   datetime64[ns, UTC]
 2   league_season  1140 non-null   int64              
 3   league         1140 non-null   object             
 4   home_team      1140 non-null   object             
 5   away_team      1140 non-null   object             
 6   home_score     1140 non-null   int64              
 7   away_score     1140 non-null   int64              
 8   home_result    1140 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(4), object(4)
memory usage: 80.3+ KB


In [48]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

In [49]:
label_map = {'LOSS': 0, 'DRAW': 1, 'WIN': 2}
df['home_result'] = df['home_result'].map(label_map)

In [50]:
df

,fixture_id,date,league_season,league,home_team,away_team,home_score,away_score,home_result
0,720746,2021-08-13 19:00:00+00:00,2021,La Liga,Valencia,Getafe,1,0,2
1,720743,2021-08-14 17:30:00+00:00,2021,La Liga,Cadiz,Levante,1,1,1
2,720742,2021-08-14 17:30:00+00:00,2021,La Liga,Mallorca,Real Betis,1,1,1
3,720739,2021-08-14 20:00:00+00:00,2021,La Liga,Alaves,Real Madrid,1,4,0
4,720744,2021-08-14 20:00:00+00:00,2021,La Liga,Osasuna,Espanyol,0,0,1
...,...,...,...,...,...,...,...,...,...
1135,1038330,2024-05-25 19:00:00+00:00,2023,La Liga,Real Madrid,Real Betis,0,0,1
1136,1038324,2024-05-26 12:00:00+00:00,2023,La Liga,Getafe,Mallorca,1,2,0
1138,1038323,2024-05-26 14:15:00+00:00,2023,La Liga,Celta Vigo,Valencia,2,2,1
1137,1038331,2024-05-26 14:15:00+00:00,2023,La Liga,Las Palmas,Alaves,1,1,1


## **DROP FEATURES**

## **RESTRUCTURE DATA**

In [51]:
home_df = df[['fixture_id','date','league_season', 'home_team', 'away_team', 'home_score','away_score']].copy()
home_df.columns = ['fixture_id', 'date', 'season','team', 'opponent','goals_for', 'goals_against']
home_df['location'] = 'home'

In [52]:
away_df = df[['fixture_id', 'date','league_season','away_team', 'home_team', 'away_score','home_score']].copy()
away_df.columns = ['fixture_id', 'date','season', 'team','opponent', 'goals_for', 'goals_against']
away_df['location'] = 'away'

In [53]:
long_df = pd.concat([home_df, away_df])
long_df = long_df.sort_values(['team', 'date']).reset_index(drop=True)
long_df

,fixture_id,date,season,team,opponent,goals_for,goals_against,location
0,720739,2021-08-14 20:00:00+00:00,2021,Alaves,Real Madrid,1,4,home
1,720749,2021-08-21 15:00:00+00:00,2021,Alaves,Mallorca,0,1,home
2,720766,2021-08-27 20:15:00+00:00,2021,Alaves,Valencia,0,3,away
3,720779,2021-09-18 19:00:00+00:00,2021,Alaves,Osasuna,0,2,home
4,720793,2021-09-22 17:30:00+00:00,2021,Alaves,Espanyol,0,1,away
...,...,...,...,...,...,...,...,...
2275,1038282,2024-05-05 14:15:00+00:00,2023,Villarreal,Celta Vigo,2,3,away
2276,1038300,2024-05-11 14:15:00+00:00,2023,Villarreal,Sevilla,3,2,home
2277,1038303,2024-05-14 20:00:00+00:00,2023,Villarreal,Girona,1,0,away
2278,1038321,2024-05-19 17:00:00+00:00,2023,Villarreal,Real Madrid,4,4,home


## **AVG GOALS**

In [54]:
(long_df.groupby('team')['goals_for'].transform(lambda x: x.shift(3).rolling(window=3, min_periods=1).mean().round(3)))

0         NaN
1         NaN
2         NaN
3       1.000
4       0.500
        ...  
2275    1.000
2276    1.333
2277    2.000
2278    2.333
2279    2.667
Name: goals_for, Length: 2280, dtype: float64

In [55]:
for N in [3, 5, 10]:

    long_df[f'avg_goals_last_{N}'] = (long_df.groupby('team')['goals_for'].transform(lambda x: x.shift(1).rolling(window=N, min_periods=1).mean().round(3)))

    long_df[f'avg_goals_against_last_{N}'] = (long_df.groupby('team')['goals_against'].transform(lambda x: x.shift(1).rolling(window=N, min_periods=1).mean().round(3)))


# long_df['avg_goals_season'] = long_df.groupby(['team','season'])['goals_made'].transform('mean')

long_df['avg_goals_season'] = (
    long_df.groupby(['team', 'season'])['goals_for'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean().round(3))
)
long_df['avg_goals_against_season'] = (
    long_df.groupby(['team', 'season'])['goals_against'].transform(lambda x: x.shift(1).expanding(min_periods=1).mean().round(3))
)


long_df.fillna(0, inplace=True)


long_df['avg_scoring_last_3'] = (long_df['avg_goals_last_3'] - long_df['avg_goals_against_last_3']).round(3)
long_df['avg_scoring_last_5'] = (long_df['avg_goals_last_5'] - long_df['avg_goals_against_last_5']).round(3)
long_df['avg_scoring_10'] = (long_df['avg_goals_last_10'] - long_df['avg_goals_against_last_10']).round(3)
long_df['avg_scoring_season'] = (long_df['avg_goals_season'] - long_df['avg_goals_against_season']).round(3)



long_df

,fixture_id,date,season,team,opponent,goals_for,goals_against,location,avg_goals_last_3,avg_goals_against_last_3,avg_goals_last_5,avg_goals_against_last_5,avg_goals_last_10,avg_goals_against_last_10,avg_goals_season,avg_goals_against_season,avg_scoring_last_3,avg_scoring_last_5,avg_scoring_10,avg_scoring_season
0,720739,2021-08-14 20:00:00+00:00,2021,Alaves,Real Madrid,1,4,home,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,720749,2021-08-21 15:00:00+00:00,2021,Alaves,Mallorca,0,1,home,1.000,4.000,1.000,4.000,1.000,4.000,1.000,4.000,-3.000,-3.000,-3.000,-3.000
2,720766,2021-08-27 20:15:00+00:00,2021,Alaves,Valencia,0,3,away,0.500,2.500,0.500,2.500,0.500,2.500,0.500,2.500,-2.000,-2.000,-2.000,-2.000
3,720779,2021-09-18 19:00:00+00:00,2021,Alaves,Osasuna,0,2,home,0.333,2.667,0.333,2.667,0.333,2.667,0.333,2.667,-2.334,-2.334,-2.334,-2.334
4,720793,2021-09-22 17:30:00+00:00,2021,Alaves,Espanyol,0,1,away,0.000,2.000,0.250,2.500,0.250,2.500,0.250,2.500,-2.000,-2.250,-2.250,-2.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,1038282,2024-05-05 14:15:00+00:00,2023,Villarreal,Celta Vigo,2,3,away,2.000,0.667,1.600,0.800,2.100,1.000,1.636,1.667,1.333,0.800,1.100,-0.031
2276,1038300,2024-05-11 14:15:00+00:00,2023,Villarreal,Sevilla,3,2,home,2.333,1.333,1.800,1.400,2.200,1.200,1.647,1.706,1.000,0.400,1.000,-0.059
2277,1038303,2024-05-14 20:00:00+00:00,2023,Villarreal,Girona,1,0,away,2.667,1.667,2.200,1.400,2.400,1.300,1.686,1.714,1.000,0.800,1.100,-0.028
2278,1038321,2024-05-19 17:00:00+00:00,2023,Villarreal,Real Madrid,4,4,home,2.000,1.667,2.200,1.200,2.200,1.200,1.667,1.667,0.333,1.000,1.000,0.000


## **POINTS**

In [56]:
long_df['points'] = np.where(long_df['goals_for'] > long_df['goals_against'], 3,1)
long_df['points'] = np.where(long_df['goals_for'] < long_df['goals_against'], 0,long_df['points'])

In [57]:
for N in [3, 5, 10]:

    long_df[f'points_last_{N}'] = (long_df.groupby('team')['points'].transform(lambda x: x.shift(1).rolling(window=N, min_periods=1).sum().round(3)))

long_df.fillna(0,inplace=True)

In [58]:
long_df

,fixture_id,date,season,team,opponent,goals_for,goals_against,location,avg_goals_last_3,avg_goals_against_last_3,...,avg_goals_season,avg_goals_against_season,avg_scoring_last_3,avg_scoring_last_5,avg_scoring_10,avg_scoring_season,points,points_last_3,points_last_5,points_last_10
0,720739,2021-08-14 20:00:00+00:00,2021,Alaves,Real Madrid,1,4,home,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0,0.0,0.0,0.0
1,720749,2021-08-21 15:00:00+00:00,2021,Alaves,Mallorca,0,1,home,1.000,4.000,...,1.000,4.000,-3.000,-3.000,-3.000,-3.000,0,0.0,0.0,0.0
2,720766,2021-08-27 20:15:00+00:00,2021,Alaves,Valencia,0,3,away,0.500,2.500,...,0.500,2.500,-2.000,-2.000,-2.000,-2.000,0,0.0,0.0,0.0
3,720779,2021-09-18 19:00:00+00:00,2021,Alaves,Osasuna,0,2,home,0.333,2.667,...,0.333,2.667,-2.334,-2.334,-2.334,-2.334,0,0.0,0.0,0.0
4,720793,2021-09-22 17:30:00+00:00,2021,Alaves,Espanyol,0,1,away,0.000,2.000,...,0.250,2.500,-2.000,-2.250,-2.250,-2.250,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,1038282,2024-05-05 14:15:00+00:00,2023,Villarreal,Celta Vigo,2,3,away,2.000,0.667,...,1.636,1.667,1.333,0.800,1.100,-0.031,0,7.0,10.0,21.0
2276,1038300,2024-05-11 14:15:00+00:00,2023,Villarreal,Sevilla,3,2,home,2.333,1.333,...,1.647,1.706,1.000,0.400,1.000,-0.059,3,6.0,7.0,20.0
2277,1038303,2024-05-14 20:00:00+00:00,2023,Villarreal,Girona,1,0,away,2.667,1.667,...,1.686,1.714,1.000,0.800,1.100,-0.028,3,6.0,10.0,22.0
2278,1038321,2024-05-19 17:00:00+00:00,2023,Villarreal,Real Madrid,4,4,home,2.000,1.667,...,1.667,1.667,0.333,1.000,1.000,0.000,1,6.0,12.0,22.0


### **STREAKS**

In [59]:
long_df['win_streak'] = 0
long_df['loss_streak'] = 0


for team, group in long_df.groupby('team'):
    win_streak = 0
    loss_streak = 0
    streaks = []

    for result in group['points']:  # Points column determines win/loss/draw
        if result == 3:  # Win
            win_streak += 1
            loss_streak = 0
        elif result == 0:  # Loss
            loss_streak += 1
            win_streak = 0
        else:  # Draw
            win_streak = 0
            loss_streak = 0

        streaks.append((win_streak, loss_streak))

    # Assign streaks back to the dataframe
    long_df.loc[group.index, 'win_streak'] = [s[0] for s in streaks]
    long_df.loc[group.index, 'loss_streak'] = [s[1] for s in streaks]

# Shift streaks to ensure they reflect the state before the current match
long_df['win_streak'] = long_df.groupby('team')['win_streak'].shift(1).fillna(0).astype(int)
long_df['loss_streak'] = long_df.groupby('team')['loss_streak'].shift(1).fillna(0).astype(int)

In [60]:
long_df['streak'] = np.where(long_df['win_streak'] > 0, long_df['win_streak'], 0)
long_df['streak'] = np.where(long_df['loss_streak'] > 0, long_df['loss_streak']*-1, long_df['streak'])


long_df.drop(columns=['win_streak','loss_streak'],inplace=True)

In [61]:
long_df

,fixture_id,date,season,team,opponent,goals_for,goals_against,location,avg_goals_last_3,avg_goals_against_last_3,...,avg_goals_against_season,avg_scoring_last_3,avg_scoring_last_5,avg_scoring_10,avg_scoring_season,points,points_last_3,points_last_5,points_last_10,streak
0,720739,2021-08-14 20:00:00+00:00,2021,Alaves,Real Madrid,1,4,home,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0,0.0,0.0,0.0,0
1,720749,2021-08-21 15:00:00+00:00,2021,Alaves,Mallorca,0,1,home,1.000,4.000,...,4.000,-3.000,-3.000,-3.000,-3.000,0,0.0,0.0,0.0,-1
2,720766,2021-08-27 20:15:00+00:00,2021,Alaves,Valencia,0,3,away,0.500,2.500,...,2.500,-2.000,-2.000,-2.000,-2.000,0,0.0,0.0,0.0,-2
3,720779,2021-09-18 19:00:00+00:00,2021,Alaves,Osasuna,0,2,home,0.333,2.667,...,2.667,-2.334,-2.334,-2.334,-2.334,0,0.0,0.0,0.0,-3
4,720793,2021-09-22 17:30:00+00:00,2021,Alaves,Espanyol,0,1,away,0.000,2.000,...,2.500,-2.000,-2.250,-2.250,-2.250,0,0.0,0.0,0.0,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,1038282,2024-05-05 14:15:00+00:00,2023,Villarreal,Celta Vigo,2,3,away,2.000,0.667,...,1.667,1.333,0.800,1.100,-0.031,0,7.0,10.0,21.0,2
2276,1038300,2024-05-11 14:15:00+00:00,2023,Villarreal,Sevilla,3,2,home,2.333,1.333,...,1.706,1.000,0.400,1.000,-0.059,3,6.0,7.0,20.0,-1
2277,1038303,2024-05-14 20:00:00+00:00,2023,Villarreal,Girona,1,0,away,2.667,1.667,...,1.714,1.000,0.800,1.100,-0.028,3,6.0,10.0,22.0,1
2278,1038321,2024-05-19 17:00:00+00:00,2023,Villarreal,Real Madrid,4,4,home,2.000,1.667,...,1.667,0.333,1.000,1.000,0.000,1,6.0,12.0,22.0,2


In [62]:
long_df.sort_values(by=['date'])

,fixture_id,date,season,team,opponent,goals_for,goals_against,location,avg_goals_last_3,avg_goals_against_last_3,...,avg_goals_against_season,avg_scoring_last_3,avg_scoring_last_5,avg_scoring_10,avg_scoring_season,points,points_last_3,points_last_5,points_last_10,streak
874,720746,2021-08-13 19:00:00+00:00,2021,Getafe,Valencia,0,1,away,0.000,0.000,...,0.000,0.000,0.0,0.0,0.000,0,0.0,0.0,0.0,0
2014,720746,2021-08-13 19:00:00+00:00,2021,Valencia,Getafe,1,0,home,0.000,0.000,...,0.000,0.000,0.0,0.0,0.000,3,0.0,0.0,0.0,0
1178,720743,2021-08-14 17:30:00+00:00,2021,Levante,Cadiz,1,1,away,0.000,0.000,...,0.000,0.000,0.0,0.0,0.000,1,0.0,0.0,0.0,0
494,720743,2021-08-14 17:30:00+00:00,2021,Cadiz,Levante,1,1,home,0.000,0.000,...,0.000,0.000,0.0,0.0,0.000,1,0.0,0.0,0.0,0
1558,720742,2021-08-14 17:30:00+00:00,2021,Real Betis,Mallorca,1,1,away,0.000,0.000,...,0.000,0.000,0.0,0.0,0.000,1,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,1038331,2024-05-26 14:15:00+00:00,2023,Las Palmas,Alaves,1,1,home,0.667,1.000,...,1.243,-0.333,-1.0,-1.4,-0.378,1,2.0,2.0,2.0,0
75,1038331,2024-05-26 14:15:00+00:00,2023,Alaves,Las Palmas,1,1,away,1.000,2.333,...,1.216,-1.333,0.0,-0.2,-0.270,1,4.0,10.0,16.0,1
721,1038323,2024-05-26 14:15:00+00:00,2023,Celta Vigo,Valencia,2,2,home,1.333,1.000,...,1.486,0.333,-0.2,-0.2,-0.297,1,6.0,9.0,16.0,2
493,1038329,2024-05-26 19:00:00+00:00,2023,Barcelona,Sevilla,2,1,away,2.333,0.000,...,1.162,2.333,1.4,1.2,0.919,3,9.0,12.0,24.0,3


### **JOINING BACK**

In [63]:
home_feats = long_df[long_df['location'] == 'home'].drop(columns=['season','goals_for','goals_against','date','points','location', 'opponent']).rename(columns={
    'team' : 'home_team',
    'avg_goals_last_3': 'home_avg_goals_last_3',
    'avg_goals_against_last_3' : 'home_avg_goals_against_last_3',
    'avg_goals_last_5' : 'home_avg_goals_last_5',
    'avg_goals_against_last_5' : 'home_avg_goals_against_last_5',
    'avg_goals_last_10' : 'home_avg_goals_last_10',
    'avg_goals_against_last_10' : 'home_avg_goals_against_last_10',
    'avg_goals_season' : 'home_avg_goals_season',
    'avg_goals_against_season' : 'home_avg_goals_against_season',
    'avg_scoring_last_3' : 'home_avg_scoring_last_3',
    'avg_scoring_last_5' : 'home_avg_scoring_last_5',
    'avg_scoring_10' : 'home_avg_scoring_last_10',
    'avg_scoring_season' : 'home_avg_scoring_season',
    'points_last_3' : 'home_points_last_3',
    'points_last_5' : 'home_points_last_5',
    'points_last_10' : 'home_points_last_10',
    'streak': 'home_streak'

})

In [64]:
away_feats = long_df[long_df['location'] == 'away'].drop(columns=['season','goals_for','goals_against','date','points','location','opponent']).rename(columns={
    'team' : 'away_team',
    'avg_goals_last_3': 'away_avg_goals_last_3',
    'avg_goals_against_last_3' : 'away_avg_goals_against_last_3',
    'avg_goals_last_5' : 'away_avg_goals_last_5',
    'avg_goals_against_last_5' : 'away_avg_goals_against_last_5',
    'avg_goals_last_10' : 'away_avg_goals_last_10',
    'avg_goals_against_last_10' : 'away_avg_goals_against_last_10',
    'avg_goals_season' : 'away_avg_goals_season',
    'avg_goals_against_season' : 'away_avg_goals_against_season',
    'avg_scoring_last_3' : 'away_avg_scoring_last_3',
    'avg_scoring_last_5' : 'away_avg_scoring_last_5',
    'avg_scoring_10' : 'away_avg_scoring_last_10',
    'avg_scoring_season' : 'away_avg_scoring_season',
    'points_last_3' : 'away_points_last_3',
    'points_last_5' : 'away_points_last_5',
    'points_last_10' : 'away_points_last_10',
    'streak': 'away_streak'
})

In [65]:
df = df.merge(home_feats, on=['fixture_id', 'home_team'], how='left')
df = df.merge(away_feats, on=['fixture_id', 'away_team'], how='left')

In [66]:
df

,fixture_id,date,league_season,league,home_team,away_team,home_score,away_score,home_result,home_avg_goals_last_3,...,away_avg_goals_season,away_avg_goals_against_season,away_avg_scoring_last_3,away_avg_scoring_last_5,away_avg_scoring_last_10,away_avg_scoring_season,away_points_last_3,away_points_last_5,away_points_last_10,away_streak
0,720746,2021-08-13 19:00:00+00:00,2021,La Liga,Valencia,Getafe,1,0,2,0.000,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0
1,720743,2021-08-14 17:30:00+00:00,2021,La Liga,Cadiz,Levante,1,1,1,0.000,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0
2,720742,2021-08-14 17:30:00+00:00,2021,La Liga,Mallorca,Real Betis,1,1,1,0.000,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0
3,720739,2021-08-14 20:00:00+00:00,2021,La Liga,Alaves,Real Madrid,1,4,0,0.000,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0
4,720744,2021-08-14 20:00:00+00:00,2021,La Liga,Osasuna,Espanyol,0,0,1,0.000,...,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1038330,2024-05-25 19:00:00+00:00,2023,La Liga,Real Madrid,Real Betis,0,0,1,4.333,...,1.297,1.216,-0.333,0.2,-0.1,0.081,4.0,8.0,14.0,-1
1136,1038324,2024-05-26 12:00:00+00:00,2023,La Liga,Getafe,Mallorca,1,2,0,0.000,...,0.838,1.162,0.333,0.0,-0.2,-0.324,5.0,6.0,10.0,0
1137,1038323,2024-05-26 14:15:00+00:00,2023,La Liga,Celta Vigo,Valencia,2,2,1,1.333,...,1.027,1.162,-1.000,-1.2,-0.6,-0.135,1.0,1.0,8.0,-2
1138,1038331,2024-05-26 14:15:00+00:00,2023,La Liga,Las Palmas,Alaves,1,1,1,0.667,...,0.946,1.216,-1.333,0.0,-0.2,-0.270,4.0,10.0,16.0,1


### **ELO**

In [67]:
initial_elo = 1500
elo_scores = {}

# Function to calculate expected outcome
def expected_outcome(elo_a, elo_b):
    return 1 / (1 + 10 ** ((elo_b - elo_a) / 400))

# Function to update ELO scores
def update_elo(elo_a, elo_b, result_a, k=20):
    expected_a = expected_outcome(elo_a, elo_b)
    new_elo_a = elo_a + k * (result_a - expected_a)
    new_elo_b = elo_b + k * ((1 - result_a) - (1 - expected_a))
    return new_elo_a, new_elo_b

# Add ELO columns to the dataframe
df['home_elo'] = 0
# df['home_elo_after'] = 0
df['away_elo'] = 0
# df['away_elo_after'] = 0

# Process matches in chronological order
for idx, row in df.iterrows():
    home_team = row['home_team']  # Current team
    away_team = row['away_team']  # Opponent team
    result = 1 if row['home_result']==2 else 0.5 if row['home_result']==1 else 0



    # Initialize ELO scores if not already set
    if home_team not in elo_scores:
        elo_scores[home_team] = initial_elo
    if away_team not in elo_scores:
        elo_scores[away_team] = initial_elo

    # Get current ELO scores
    elo_a = elo_scores[home_team]
    elo_b = elo_scores[away_team]

    # Update ELO scores
    new_elo_a, new_elo_b = update_elo(elo_a, elo_b, result)

    # Save ELO scores before and after the match
    df.loc[idx, 'home_elo'] = round(elo_a,2)
    # df.loc[idx, 'home_elo_after'] = round(new_elo_a,2)
    df.loc[idx, 'away_elo'] = round(elo_b,2)
    # df.loc[idx, 'away_elo_after'] = round(new_elo_b,2)

    # Update the ELO scores in the dictionary
    elo_scores[home_team] = round(new_elo_a,2)
    elo_scores[away_team] = round(new_elo_b,2)

C:\Users\danie\AppData\Local\Temp\ipykernel_23572\1305377426.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1509.71' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'home_elo'] = round(elo_a,2)
C:\Users\danie\AppData\Local\Temp\ipykernel_23572\1305377426.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1480.29' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'away_elo'] = round(elo_b,2)


In [68]:
df

,fixture_id,date,league_season,league,home_team,away_team,home_score,away_score,home_result,home_avg_goals_last_3,...,away_avg_scoring_last_3,away_avg_scoring_last_5,away_avg_scoring_last_10,away_avg_scoring_season,away_points_last_3,away_points_last_5,away_points_last_10,away_streak,home_elo,away_elo
0,720746,2021-08-13 19:00:00+00:00,2021,La Liga,Valencia,Getafe,1,0,2,0.000,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,1500.00,1500.00
1,720743,2021-08-14 17:30:00+00:00,2021,La Liga,Cadiz,Levante,1,1,1,0.000,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,1500.00,1500.00
2,720742,2021-08-14 17:30:00+00:00,2021,La Liga,Mallorca,Real Betis,1,1,1,0.000,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,1500.00,1500.00
3,720739,2021-08-14 20:00:00+00:00,2021,La Liga,Alaves,Real Madrid,1,4,0,0.000,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,1500.00,1500.00
4,720744,2021-08-14 20:00:00+00:00,2021,La Liga,Osasuna,Espanyol,0,0,1,0.000,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,1500.00,1500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1038330,2024-05-25 19:00:00+00:00,2023,La Liga,Real Madrid,Real Betis,0,0,1,4.333,...,-0.333,0.2,-0.1,0.081,4.0,8.0,14.0,-1,1742.55,1547.01
1136,1038324,2024-05-26 12:00:00+00:00,2023,La Liga,Getafe,Mallorca,1,2,0,0.000,...,0.333,0.0,-0.2,-0.324,5.0,6.0,10.0,0,1455.25,1458.68
1137,1038323,2024-05-26 14:15:00+00:00,2023,La Liga,Celta Vigo,Valencia,2,2,1,1.333,...,-1.000,-1.2,-0.6,-0.135,1.0,1.0,8.0,-2,1464.76,1473.46
1138,1038331,2024-05-26 14:15:00+00:00,2023,La Liga,Las Palmas,Alaves,1,1,1,0.667,...,-1.333,0.0,-0.2,-0.270,4.0,10.0,16.0,1,1443.37,1463.76


In [69]:
df['scoring_diff_last_3'] = (df['home_avg_scoring_last_3'] - df['away_avg_scoring_last_3']).round(2)
df['scoring_diff_last_5'] = (df['home_avg_scoring_last_5'] - df['away_avg_scoring_last_5']).round(2)
df['scoring_diff_last_10'] = (df['home_avg_scoring_last_10'] - df['away_avg_scoring_last_10']).round(2)
df['scoring_diff_season'] = (df['home_avg_scoring_season'] - df['away_avg_scoring_season']).round(2)

df['points_diff_last_3'] = (df['home_points_last_3'] - df['away_points_last_3']).round(2)
df['points_diff_last_5'] = (df['home_points_last_5'] - df['away_points_last_5']).round(2)
df['points_diff_last_10'] = (df['home_points_last_10'] - df['away_points_last_10']).round(2)


df['streak_diff'] = (df['home_streak'] - df['away_streak']).round(2)
df['elo_diff'] = (df['home_elo'] - df['away_elo']).round(2)

df


,fixture_id,date,league_season,league,home_team,away_team,home_score,away_score,home_result,home_avg_goals_last_3,...,away_elo,scoring_diff_last_3,scoring_diff_last_5,scoring_diff_last_10,scoring_diff_season,points_diff_last_3,points_diff_last_5,points_diff_last_10,streak_diff,elo_diff
0,720746,2021-08-13 19:00:00+00:00,2021,La Liga,Valencia,Getafe,1,0,2,0.000,...,1500.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,0.00
1,720743,2021-08-14 17:30:00+00:00,2021,La Liga,Cadiz,Levante,1,1,1,0.000,...,1500.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,0.00
2,720742,2021-08-14 17:30:00+00:00,2021,La Liga,Mallorca,Real Betis,1,1,1,0.000,...,1500.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,0.00
3,720739,2021-08-14 20:00:00+00:00,2021,La Liga,Alaves,Real Madrid,1,4,0,0.000,...,1500.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,0.00
4,720744,2021-08-14 20:00:00+00:00,2021,La Liga,Osasuna,Espanyol,0,0,1,0.000,...,1500.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1038330,2024-05-25 19:00:00+00:00,2023,La Liga,Real Madrid,Real Betis,0,0,1,4.333,...,1547.01,3.33,2.4,2.4,1.57,3.0,5.0,14.0,1,195.54
1136,1038324,2024-05-26 12:00:00+00:00,2023,La Liga,Getafe,Mallorca,1,2,0,0.000,...,1458.68,-2.00,-1.0,-0.4,0.03,-5.0,-3.0,-2.0,-4,-3.43
1137,1038323,2024-05-26 14:15:00+00:00,2023,La Liga,Celta Vigo,Valencia,2,2,1,1.333,...,1473.46,1.33,1.0,0.4,-0.16,5.0,8.0,8.0,4,-8.70
1138,1038331,2024-05-26 14:15:00+00:00,2023,La Liga,Las Palmas,Alaves,1,1,1,0.667,...,1463.76,1.00,-1.0,-1.2,-0.11,-2.0,-8.0,-14.0,-1,-20.39


## **HEAD 2 HEAD**

In [70]:
# from collections import defaultdict

# history = defaultdict(list)

# history

In [71]:
from collections import defaultdict

# Sort matches by date
df = df.sort_values('date').reset_index(drop=True)

# Initialize storage for H2H features
h2h_features = {
    'fixture_id': [],
    'h2h_matches': [],
    'h2h_home_wins': [],
    'h2h_away_wins': [],
    'h2h_draws': [],
    'h2h_home_goals': [],
    'h2h_away_goals': [],
}

N = 5  # Last N head-to-head matches

# Historical storage
history = defaultdict(list)

# Loop over each match
for _, row in df.iterrows():
    home = row['home_team']
    away = row['away_team']
    date = row['date']
    fixture_id = row['fixture_id']

    # Define a key that ignores home/away for H2H tracking
    matchup_key = tuple(sorted([home, away]))

    # Get past N matches before this one
    past_matches = [m for m in history[matchup_key] if m['date'] < date]
    past_matches = sorted(past_matches, key=lambda x: x['date'])[-N:]

    # Extract stats
    h2h_matches = len(past_matches)
    home_wins = away_wins = draws = 0
    home_goals = away_goals = 0

    for m in past_matches:
        if m['home'] == home:
            home_goals += m['home_score']
            away_goals += m['away_score']
            if m['home_score'] > m['away_score']:
                home_wins += 1
            elif m['home_score'] < m['away_score']:
                away_wins += 1
            else:
                draws += 1
        else:
            home_goals += m['away_score']
            away_goals += m['home_score']
            if m['away_score'] > m['home_score']:
                home_wins += 1
            elif m['away_score'] < m['home_score']:
                away_wins += 1
            else:
                draws += 1

    # Store features
    h2h_features['fixture_id'].append(fixture_id)
    h2h_features['h2h_matches'].append(h2h_matches)
    h2h_features['h2h_home_wins'].append(home_wins)
    h2h_features['h2h_away_wins'].append(away_wins)
    h2h_features['h2h_draws'].append(draws)
    h2h_features['h2h_home_goals'].append(home_goals)
    h2h_features['h2h_away_goals'].append(away_goals)

    # Update history
    history[matchup_key].append({
        'date': date,
        'home': home,
        'away': away,
        'home_score': row['home_score'],
        'away_score': row['away_score']
    })

# Convert to DataFrame and merge
h2h_df = pd.DataFrame(h2h_features)
df = df.merge(h2h_df, on='fixture_id', how='left')

In [72]:
df.drop(columns='h2h_matches')
df['h2h_goal_diff'] = (df['h2h_home_goals'] - df['h2h_away_goals']).round(2)

In [73]:
# from collections import defaultdict

# # Sort matches by date
# df = df.sort_values('date').reset_index(drop=True)

# # Initialize storage
# h2h_features = {
#     'fixture_id': [],
#     'h2h_matches': [],
#     'h2h_home_wins': [],
#     'h2h_away_wins': [],
#     'h2h_draws': [],
#     'h2h_home_goals': [],
#     'h2h_away_goals': [],
# }

# N = 5  # Last N head-to-head matches

# # Historical storage
# history = defaultdict(list)

# # Loop over each match
# for _, row in df.iterrows():
#     home = row['home_team']
#     away = row['away_team']
#     date = row['date']
#     fixture_id = row['fixture_id']

#     # Define a key that ignores home/away for H2H tracking
#     matchup_key = tuple(sorted([home, away]))

#     # Get past N matches before this one
#     past_matches = [m for m in history[matchup_key] if m['date'] < date]
#     past_matches = sorted(past_matches, key=lambda x: x['date'])[-N:]

#     # Extract stats
#     h2h_matches = len(past_matches)
#     home_wins = away_wins = draws = 0
#     home_goals = away_goals = 0

#     for m in past_matches:
#         if m['home'] == home:
#             home_goals += m['home_score']
#             away_goals += m['away_score']
#             if m['home_score'] > m['away_score']:
#                 home_wins += 1
#             elif m['home_score'] < m['away_score']:
#                 away_wins += 1
#             else:
#                 draws += 1
#         else:
#             home_goals += m['away_score']
#             away_goals += m['home_score']
#             if m['away_score'] > m['home_score']:
#                 home_wins += 1
#             elif m['away_score'] < m['home_score']:
#                 away_wins += 1
#             else:
#                 draws += 1

#     # Store features
#     h2h_features['fixture_id'].append(fixture_id)
#     h2h_features['h2h_matches'].append(h2h_matches)
#     h2h_features['h2h_home_wins'].append(home_wins)
#     h2h_features['h2h_away_wins'].append(away_wins)
#     h2h_features['h2h_draws'].append(draws)
#     h2h_features['h2h_home_goals'].append(home_goals)
#     h2h_features['h2h_away_goals'].append(away_goals)

#     # Update history
#     history[matchup_key].append({
#         'date': date,
#         'home': home,
#         'away': away,
#         'home_score': row['home_score'],
#         'away_score': row['away_score']
#     })

# # # Convert to DataFrame and merge
# # h2h_df = pd.DataFrame(h2h_features)
# # df = df.merge(h2h_df, on='fixture_id', how='left')

# history[matchup_key]

In [74]:
df.to_csv('../data/processed.csv', index=False)

In [75]:
df[10:-140].to_csv('../data/train.csv', index=False)


In [76]:
df[-140:].to_csv('../data/test.csv', index=False)

In [77]:
df[10:]

,fixture_id,date,league_season,league,home_team,away_team,home_score,away_score,home_result,home_avg_goals_last_3,...,points_diff_last_10,streak_diff,elo_diff,h2h_matches,h2h_home_wins,h2h_away_wins,h2h_draws,h2h_home_goals,h2h_away_goals,h2h_goal_diff
10,720752,2021-08-20 19:00:00+00:00,2021,La Liga,Real Betis,Cadiz,1,1,1,1.000,...,0.0,0,0.00,0,0,0,0,0,0,0
11,720749,2021-08-21 15:00:00+00:00,2021,La Liga,Alaves,Mallorca,0,1,0,1.000,...,-1.0,-1,-10.00,0,0,0,0,0,0,0
12,720755,2021-08-21 17:30:00+00:00,2021,La Liga,Espanyol,Villarreal,0,0,1,0.000,...,0.0,0,0.00,0,0,0,0,0,0,0
13,720754,2021-08-21 17:30:00+00:00,2021,La Liga,Granada CF,Valencia,1,1,1,0.000,...,-2.0,-1,-10.00,0,0,0,0,0,0,0
14,720750,2021-08-21 20:00:00+00:00,2021,La Liga,Athletic Club,Barcelona,1,1,1,0.000,...,-2.0,-1,-10.00,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1038330,2024-05-25 19:00:00+00:00,2023,La Liga,Real Madrid,Real Betis,0,0,1,4.333,...,14.0,1,195.54,5,2,0,3,4,2,2
1136,1038324,2024-05-26 12:00:00+00:00,2023,La Liga,Getafe,Mallorca,1,2,0,0.000,...,-2.0,-4,-3.43,5,2,1,2,4,3,1
1137,1038331,2024-05-26 14:15:00+00:00,2023,La Liga,Las Palmas,Alaves,1,1,1,0.667,...,-14.0,-1,-20.39,1,1,0,0,1,0,1
1138,1038323,2024-05-26 14:15:00+00:00,2023,La Liga,Celta Vigo,Valencia,2,2,1,1.333,...,8.0,4,-8.70,5,0,4,1,2,9,-7
